### (事前準備: OpenAI APIキーの設定)
OpenAI APIを使う為のAPIキーを設定します  
このAPIキーについては、OpenAIのサイトで取得することが出来ます  
https://platform.openai.com/api-keys  
APIキーについては公開しないように注意してください  

下記のリンクをクリックするとGoogle Colabで実行することが出来ます  
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/2Nike2/LangChainPractice/blob/main/00_03_use_retriever.ipynb)

In [ ]:
import os

# ここにあなたのOpenAIのAPIキーを入力してください
openai_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = openai_api_key


### (事前準備: Tavily APIキーの設定)
AIエージェントに適した、Tavilyの検索エンジンのAPIキーを設定します  
https://app.tavily.com/home  
APIキーについては公開しないように注意してください  


In [ ]:
# ここにあなたのOpenAIのAPIキーを入力してください
tavily_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('TAVILY_API_KEY') is None:
    os.environ['TAVILY_API_KEY'] = tavily_api_key

## エージェント
LLMが使用出来るツールを登録して機能を拡張します
ここでは予めダウンロードしてきた文書を検査するツールと、
上記とは別にWebで動的に検索するツールを追加します

### ライブラリのインストール
LangChainのライブラリをインストールします  
またWeb文書を取得するためのライブラリであるBeautifulSoup  
及びベクトルデータベースのfaiss(CPU版)もインストールします  


In [ ]:
!pip install langchain==0.1.4
!pip install langchain-openai==0.0.5
!pip install beautifulsoup4==4.12.3
!pip install faiss-cpu==1.7.4
!pip install langchainhub==0.1.14


### モデルの初期化
OpenAI APIを使う為のモデルを初期化します

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)


### Web文書の取得
WebBaseLoaderを使ってWeb文書を取得します

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.aozora.gr.jp/cards/000081/files/43754_17659.html") # 青空文庫 「注文の多い料理店」(宮沢賢治)

docs = loader.load()


### 埋め込みモデル
埋め込みモデルを初期化します
これは今まで使ってきた文章に対して文章を返すモデルと違い、文章に対して埋め込みベクトル(数百~数千個の数値のリスト)を返すモデルです  
個の埋め込みベクトルというのは文章の意味を数値に要約したものと考えられ、複数の埋め込みベクトルの距離や角度を計算することによって文章の意味の近さを捉えられます  
これを利用して指示や質問に対して意味が近かったり関連性が高い文章を探し出すことが出来ます

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


### ベクトルデータベースの用意
上記の埋め込みベクトルを格納して高速に検索するためのベクトルデータベースを用意します  
ベクトルデータベースに登録する文章は全体をそのまま入れるのではなく、何らかの単位(字数、章、ページ区切り等)で分割してからベクトル化して保存することになります

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter()
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=800, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


### リトリーバの作成

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={'k': 3})


## ツールの登録
ツールを登録します

### 取得済みWeb文書リトリーバ

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 
                                       'miyazawakenji_restaulant',
                                       '宮沢賢治の「注文の多い料理店」について調べるツールです。宮沢賢治の「注文の多い料理店」に関する質問の場合はこのツールを使って答えて下さい。')


### AI検索ツール

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()


## エージェントの作成

In [ ]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

tools = [retriever_tool, search]

prompt = hub.pull('hwchase17/openai-functions-agent')
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### 動作確認

In [ ]:
agent_executor.invoke({'input': '玄関の札に出ていたレストランの日本語名は？'})


In [ ]:
agent_executor.invoke({"input": "現在のアメリカの大統領の名前は？"})
